<a href="https://colab.research.google.com/github/takzen/ai-engineering-handbook/blob/main/notebooks/030_Sklearn_Pipelines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<a href="https://colab.research.google.com/github/takzen/ai-engineering-handbook/blob/main/30_Sklearn_Pipelines.ipynb" target="_parent">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


# ⛓️ Sklearn Pipelines: Budowa Fabryki AI

Do tej pory traktowaliśmy każdy krok (czyszczenie, skalowanie, trening) jako osobną operację.
To proszenie się o kłopoty (błędy, wyciek danych).

**Pipeline (Rurociąg)** to obiekt, który łączy te kroki w łańcuch.
Działa jak taśma produkcyjna w fabryce Toyoty:
*   Wejście: Brudne dane.
*   Krok 1: Wypełnij braki (Imputer).
*   Krok 2: Przeskaluj liczby (Scaler).
*   Krok 3: Wybierz ważne cechy (Selector).
*   Krok 4: Wytrenuj model (Classifier).
*   Wyjście: Gotowa predykcja.

Dzięki temu, gdy wywołujesz `pipeline.fit()`, dzieje się to wszystko naraz.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# 1. TWORZYMY TRUDNE DANE (Z brakami i różną skalą)
data = pd.DataFrame({
    'Wiek': [25, 30, np.nan, 40, 50, 22, np.nan, 45],     # Braki!
    'Zarobki': [50000, 70000, 60000, 120000, np.nan, 45000, 80000, 95000], # Duże liczby!
    'Liczba_Dzieci': [0, 1, 2, 3, 0, 0, 1, 2],            # Małe liczby
    'Szum_1': np.random.rand(8),                          # Śmieci
    'Szum_2': np.random.rand(8),                          # Śmieci
    'Kupil': [0, 0, 1, 1, 1, 0, 1, 1]                     # Target
})

X = data.drop('Kupil', axis=1)
y = data['Kupil']

# Podział (standardowy)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

print("--- DANE SUROWE (Zauważ NaN i różne rzędy wielkości) ---")
display(X_train.head())

--- DANE SUROWE (Zauważ NaN i różne rzędy wielkości) ---


,Wiek,Zarobki,Liczba_Dzieci,Szum_1,Szum_2
0,25.0,50000.0,0,0.492995,0.471989
7,45.0,95000.0,2,0.956345,0.618779
2,NaN,60000.0,2,0.720844,0.489820
4,50.0,NaN,0,0.218592,0.258003
3,40.0,120000.0,3,0.222449,0.816059


## Budujemy Rurociąg

Zdefiniujemy listę kroków.
Każdy krok to krotka: `('nazwa', Obiekt)`.

Zauważ, że **nie musimy** teraz robić `fit_transform` ręcznie dla każdego etapu. Pipeline zrobi to sam.

In [2]:
# Definicja Pipeline'a
moj_pipeline = Pipeline([
    # Krok 1: Łatanie dziur (Wiek i Zarobki dostaną średnią)
    ('imputer', SimpleImputer(strategy='mean')),
    
    # Krok 2: Skalowanie (Żeby Zarobki nie zdominowały Wieku)
    ('scaler', StandardScaler()),
    
    # Krok 3: Wybór cech (Chcemy 3 najlepsze kolumny, wyrzucamy szum)
    ('selector', SelectKBest(score_func=f_classif, k=3)),
    
    # Krok 4: Model (Mózg operacji)
    ('model', RandomForestClassifier(random_state=42))
])

print("✅ Pipeline zdefiniowany. Jeszcze nie wytrenowany.")
# Możemy go nawet wyświetlić graficznie w Colabie!
from sklearn import set_config
set_config(display='diagram')
display(moj_pipeline)

✅ Pipeline zdefiniowany. Jeszcze nie wytrenowany.


,steps,"[('imputer', ...), ('scaler', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,missing_values,nan
,strategy,'mean'
,fill_value,None
,copy,True
,add_indicator,False
,keep_empty_features,False
,copy,True


## Trening i Predykcja (Jedna linijka!)

Teraz dzieje się magia.
Wpisujemy `fit` tylko raz. Pipeline automatycznie przesyła dane przez rury.

Co ważniejsze: przy `predict` (na danych testowych), Pipeline pamięta średnie i odchylenia z treningu!
Nie popełni błędu **Data Leakage** (nie będzie podglądał danych testowych, żeby policzyć średnią).

In [3]:
# 1. TRENING (Wszystkie 4 kroki naraz)
moj_pipeline.fit(X_train, y_train)

# 2. PREDYKCJA
# Pipeline sam uzupełni braki w X_test, przeskaluje je itd.
y_pred = moj_pipeline.predict(X_test)

print(f"Predykcje: {y_pred}")
print(f"Prawdziwe: {y_test.values}")
print(f"Dokładność: {accuracy_score(y_test, y_pred)*100:.0f}%")

Predykcje: [1 0]
Prawdziwe: [0 0]
Dokładność: 50%


## Dostęp do wnętrza Pipeline'a

A co jeśli chcemy zobaczyć, jakie cechy wybrał `SelectKBest` w 3. kroku?
Możemy dostać się do każdego elementu rurociągu po nazwie.

In [4]:
# Wyciągamy krok o nazwie 'selector'
selector_step = moj_pipeline.named_steps['selector']

# Sprawdzamy, które kolumny wybrał (zwraca maskę True/False)
wybrane_maska = selector_step.get_support()

# Nazwy kolumn (musimy je wyciągnąć, bo Imputer usuwa nagłówki, ale my wiemy jaka była kolejność)
wszystkie_cechy = X.columns
wybrane_cechy = wszystkie_cechy[wybrane_maska]

print("--- CO DZIAŁO SIĘ W ŚRODKU? ---")
print(f"Selector odrzucił 2 kolumny (szum).")
print(f"Wybrane cechy: {list(wybrane_cechy)}")

--- CO DZIAŁO SIĘ W ŚRODKU? ---
Selector odrzucił 2 kolumny (szum).
Wybrane cechy: ['Wiek', 'Zarobki', 'Liczba_Dzieci']


## 🧠 Podsumowanie: Dlaczego Profesjonaliści używają Pipeline?

1.  **Czystość kodu:** Zamiast 50 linii kodu z `fit_transform`, masz jedną definicję.
2.  **Bezpieczeństwo:** Nie ma ryzyka, że zapomnisz przeskalować danych testowych. Pipeline robi to automatcznie.
3.  **Łatwy GridSearch:** Możesz optymalizować parametry całego rurociągu naraz!
    *   Możesz sprawdzić: *"Czy lepiej użyć `strategy='mean'` czy `'median'` w Imputerze?"*
    *   Możesz sprawdzić: *"Czy lepiej wybrać `k=2` czy `k=4` cechy?"*

Pipeline to fundament **ML Ops**. Takie obiekty zapisuje się do pliku (`joblib`) i wysyła na produkcję.